In [3]:
import pandas as pd
import re
from urllib.parse import urlparse

In [24]:
# Read the Sales Data file
df_SalesData = pd.read_csv('/Users/daan/Desktop/HVA/Database management/salesfinal1.csv',  sep=None, engine='python', encoding='ISO-8859-1')
df_NMovieTable = pd.read_csv('/Users/daan/Desktop/HVA/Database management/NewMovieTable 1.csv', sep=None, engine='python', encoding='ISO-8859-1')

# View the columns to check if it works
#print(df1.columns)

# Drop unnecessary columns from SalesData
df_SalesData = df_SalesData.drop(columns=['year', 'international_box_office', 'domestic_box_office', 'worldwide_box_office', 'release_date', 'Unnamed: 8', 'opening_weekend', 'theatre_count', 'avg run per theatre',
       'runtime', 'production_budget', 'keywords', 'creative_type', 'url'])

def clean_text(text):
    if pd.isna(text):  # Check if the text is NaN
        return ''  # Return an empty string or handle NaN as needed
    # Convert text to lowercase
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Apply the cleaning function to the 'title' column
df_SalesData['title'] = df_SalesData['title'].apply(clean_text)

df_SalesData['normalized_title'] = df_SalesData['title'].apply(clean_text)
df_NMovieTable['normalized_title'] = df_NMovieTable['TITLE'].apply(clean_text)

df_SalesData = df_SalesData.merge(df_NMovieTable[['normalized_title', 'Movie_ID']], 
                                             on='normalized_title', how='left')


df_SalesData = df_SalesData.drop_duplicates()



# Handle NaN values in the 'genre' column by excluding them from the ID assignment
# We'll only assign IDs to rows where 'genre' is not NaN
df_SalesData['genre_id'] = (
    df_SalesData['genre']
    .where(df_SalesData['genre'].notna())  # Keep NaN as NaN
    .groupby(df_SalesData['genre'])
    .ngroup()  # Assign unique integers to each genre group
    .astype('Int64')  # Ensures that NaN is retained and non-NaN values are integers
)

# Convert non-NaN values to string and pad with leading zeros, leave NaN as it is
df_SalesData['genre_id'] = (
    df_SalesData['genre_id']
    #.apply(lambda x: str(x).zfill(4) if pd.notna(x) else x)  # Format with leading zeros
    #.apply(lambda x: generate_genre_id() if pd.notna(x) else None)
    #.apply(lambda x: 'g' + str(x).zfill(4) if pd.notna(x) else x)  # Prefix 'g' and format with leading zeros
    .apply(
    lambda x: f"g{int(x):02}" if pd.notna(x) else x  # Ensures 'g' prefix and removes .0
    )
)

# Print the columns and the final DataFrame
print(df_SalesData[['title', 'genre', 'genre_id']].head())

                     title   genre genre_id
0             bakha satang   Drama      g06
1                 santitos     NaN      NaN
2  chiedimi se sono felice  Comedy      g03
3              raju chacha     NaN      NaN
4    shadow of the vampire   Drama      g06


In [25]:
print(df_SalesData['title'].apply(type).value_counts())

title
<class 'str'>    31358
Name: count, dtype: int64


In [26]:
Genre_Sales = df_SalesData[df_SalesData['Movie_ID'].isna()]

print("Unmatched rows in df_SalesData:")
print(Genre_Sales[['title', 'normalized_title']])

Unmatched rows in df_SalesData:
Empty DataFrame
Columns: [title, normalized_title]
Index: []


In [27]:
print(df_SalesData.columns)

Index(['title', 'genre', 'normalized_title', 'Movie_ID', 'genre_id'], dtype='object')


In [28]:
df_Genre_Sales = df_SalesData[['genre_id', 'genre','Movie_ID']]
print(df_Genre_Sales.head())

  genre_id   genre  Movie_ID
0      g06   Drama         1
1      NaN     NaN         3
2      g03  Comedy     17328
3      NaN     NaN     17329
4      g06   Drama     17330


In [31]:
import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='DBM',
    user='postgres',
    password='Dk141621?18!',
    host='localhost',
    port='5432'
)

# Create a cursor object
cursor = conn.cursor()

# Define the table creation query
create_table_query = """
CREATE TABLE IF NOT EXISTS Genre (
    genre_ID INTEGER PRIMARY KEY,
    genre TEXT NOT NULL,
    Movie_ID INTEGER NOT NULL
);
"""

# Execute the table creation query
cursor.execute(create_table_query)

# Commit the changes
conn.commit()

# Define the insert query
insert_query = """
INSERT INTO Genre (genre_ID, genre, Movie_ID)
VALUES (%s, %s, %s)
ON CONFLICT (genre_ID) DO UPDATE
SET genre = EXCLUDED.genre,
    Movie_ID = EXCLUDED.Movie_ID;
"""

# Insert data into the table
for index, row in df_Genre.iterrows():
    cursor.execute(insert_query, (row['genre_id'], row['genre'], row['Movie_ID']))

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
